# Тестирование Spell Checker

В этом ноутбуке будет произведено базовое тестировние написанной модели.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import gc
import sys
import os
import json
import re
from string import punctuation
sys.path.append('..')

import dotenv
import numpy as np
import pandas as pd
from transformers import BertForMaskedLM, BertTokenizer, BertConfig

from deeppavlov.core.data.simple_vocab import SimpleVocabulary

import kenlm
from sacremoses import MosesTokenizer, MosesDetokenizer

from src.models.SpellChecker import *
from src.models.BertScorer.bert_scorer_correction import (
    BertScorerCorrection
)

from IPython.display import display
from tqdm.notebook import tqdm

[nltk_data] Downloading package punkt to /home/mrgeekman/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/mrgeekman/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package perluniprops to
[nltk_data]     /home/mrgeekman/nltk_data...
[nltk_data]   Package perluniprops is already up-to-date!
[nltk_data] Downloading package nonbreaking_prefixes to
[nltk_data]     /home/mrgeekman/nltk_data...
[nltk_data]   Package nonbreaking_prefixes is already up-to-date!


In [3]:
PROJECT_PATH = os.path.join(os.path.abspath(''), os.pardir)
DATA_PATH = os.path.join(PROJECT_PATH, 'data')
MODEL_PATH = os.path.join(PROJECT_PATH, 'models')

## Инициализация

Начнем с того, что инициализируем все необходимые компоненты модели. Параллельно так же будет описана роль каждого компонента в системе.

In [ ]:
raw_tokenizer = MosesTokenizer(lang='ru')
raw_detokenizer = MosesDetokenizer(lang='ru')
tokenizer = lambda x: raw_tokenizer.tokenize(x, escape=False)
detokenizer = lambda x: raw_detokenizer.detokenize(x)

In [ ]:
vocab_path = os.path.join(DATA_PATH, 'external', 'russian_words', 
                          'russian_words_vocab.dict')
vocab = SimpleVocabulary(load_path=vocab_path, save_path=vocab_path)
handcode_table_path = os.path.join(DATA_PATH, 'processed', 'handcode_table', 
                                   'table.json')
with open(handcode_table_path, 'r') as inf:
    handcode_table = json.load(inf)
candidate_generator = CandidateGenerator(
    words=vocab.keys(), handcode_table=handcode_table, max_distance=1
)

In [ ]:
model_left_right = kenlm.LanguageModel(
    os.path.join(MODEL_PATH, 'kenlm', 'left_right_3_100.arpa.binary')
)
model_right_left = kenlm.LanguageModel(
    os.path.join(MODEL_PATH, 'kenlm', 'right_left_3_100.arpa.binary')
)
position_selector = KenlmPositionSelector(model_left_right, model_right_left)

In [ ]:
BERT_PATH = os.path.join(MODEL_PATH, 'conversational_rubert')
config = BertConfig.from_json_file(
    os.path.join(BERT_PATH, 'bert_config.json')
)
model = BertForMaskedLM.from_pretrained(
    os.path.join(BERT_PATH, 'pytorch_model.bin'),
    config=config
)
bert_tokenizer = BertTokenizer(os.path.join(BERT_PATH, 'vocab.txt'))
bert_scorer_correction = BertScorerCorrection(model, bert_tokenizer)
agg_subtoken_func = np.mean
bert_scorer = BertScorer(
    bert_scorer_correction, agg_subtoken_func
)
candidate_scorer = CandidateScorer(bert_scorer)

In [ ]:
stopping_criteria = MarginStoppingCriteria(np.log(2.5))

In [ ]:
# количество кандидатов, отбираемых position selector
num_selected_candidates = 5
# максимальное количество итераций
max_it = 5

spellchecker = IterativeSpellChecker(
    candidate_generator,
    position_selector,
    candidate_scorer,
    stopping_criteria,
    tokenizer,
    detokenizer,
    num_selected_candidates=num_selected_candidates,
    max_it=max_it
)

## Тестирование

### Обучающая выборка

In [ ]:
with open(
    os.path.join(DATA_PATH, 'external', 'spell_ru_eval', 'train_source.txt'), 
    'r'
) as inf:
    sentences = inf.readlines()
    
with open(
    os.path.join(DATA_PATH, 'external', 'spell_ru_eval', 
                 'train_corrected.txt'), 
    'r'
) as inf:
    true_sentences = inf.readlines()

Запустим наш spell checker, подавая ему предложения батчами размера `batch_size`.

In [ ]:
batch_size = 5
sentences_corrected = []
num_batches = int(np.ceil(len(sentences) // batch_size))

for i in tqdm(range(num_batches)):
    cur_sentences = sentences[i*batch_size:(i+1)*batch_size]
    sentences_corrected += spellchecker(cur_sentences)

Запишем результаты файл.

In [ ]:
!mkdir -p ../data/processed/results_base/

In [ ]:
with open(os.path.join(DATA_PATH, 'processed', 'results_base', 'train.txt'), 'w') as ouf:
    ouf.writelines([sentence + '\n' for sentence in sentences_corrected])

Выполним скрит для измерения качества.

In [4]:
!python ../src/evaluation/spell_ru_eval/evaluate.py -d ../data/processed/results_base/diffs_train.txt ../data/external/spell_ru_eval/train_source.txt ../data/external/spell_ru_eval/train_corrected.txt ../data/processed/results_base/train.txt | tail -n 4

Precision=62.22, Interval=(59.98, 64.45)
Recall=65.32, Interval=(62.97, 67.66)
FMeasure=63.73, Interval=(61.44, 66.02)
1128 1813 1727


Как видим, имеем
* True Positive: $1128$
* Внесенных исправлений: $1813$ 
* Требуемых исправления: $1727$ 
* Precision: $62.22$, доверительный интервал: $(59.98, 64.45)$
* Recall: $65.32$, доверительный интервал: $(62.97, 67.66)$
* FMeasure: $63.73$, доверительный интервал: $(61.44, 66.02)$

### Тестовая выборка

In [ ]:
with open(
    os.path.join(DATA_PATH, 'external', 'spell_ru_eval', 'test_source.txt'), 
    'r'
) as inf:
    sentences = inf.readlines()
    
with open(
    os.path.join(DATA_PATH, 'external', 'spell_ru_eval', 
                 'test_corrected.txt'), 
    'r'
) as inf:
    true_sentences = inf.readlines()

Запустим наш spell checker, подавая ему предложения батчами размера `batch_size`.

In [ ]:
batch_size = 5
sentences_corrected = []
num_batches = int(np.ceil(len(sentences) // batch_size))

for i in tqdm(range(num_batches)):
    cur_sentences = sentences[i*batch_size:(i+1)*batch_size]
    sentences_corrected += spellchecker(cur_sentences)

Запишем результаты файл.

In [ ]:
with open(os.path.join(DATA_PATH, 'processed', 'results_base', 'test.txt'), 'w') as ouf:
    ouf.writelines([sentence + '\n' for sentence in sentences_corrected])

Выполним скрит для измерения качества.

In [5]:
!python ../src/evaluation/spell_ru_eval/evaluate.py -d ../data/processed/results_base/diffs_test.txt ../data/external/spell_ru_eval/test_source.txt ../data/external/spell_ru_eval/test_corrected.txt ../data/processed/results_base/test.txt | tail -n 4

Precision=52.86, Interval=(50.80, 54.93)
Recall=59.94, Interval=(57.60, 62.28)
FMeasure=56.18, Interval=(53.98, 58.38)
1182 2236 1972


Как видим, имеем
* True Positive: $1182$
* Внесенных исправлений: $2194$ 
* Требуемых исправления: $1972$ 
* Precision: $52.86$, доверительный интервал: $(50.80, 54.93)$
* Recall: $59.94$, доверительный интервал: $(57.60, 62.28)$
* FMeasure: $56.18$, доверительный интервал: $(53.98, 58.38)$

Видим, что на тесте качество достаточно ощутимо падает.

## Выводы

1. Пока результат выглядит не очень хорошо, особенно смущает precision. Казалось, что BERT сможет более качественно выполнять исправления. Возможно, эта проблема решится если использовать какую-то модель ошибок и скоры из position selector для окончательного выбора кандидата.
2. Пробовал заменить модель в position selector на 3-граммную модель, которая обучалась без текстов из соц. сетей &mdash; результат примерно такой же.
3. Пробовал заменить модель в position selector на 4-граммную модель &mdash; результат примерно такой же.
4. Пробовал увеличить максимальное расстояние Дамера-Левенштейна до двух &mdash; результат стал сильно хуже.
5. Пробовал изменить аггрегирующую функцию по WordPiece-токенам в candidate scorer &mdash; результат стал немного хуже.